# TrendMiner Solutions Training — OEE Reporting (Hands-On Notebook)
**Date:** 2025-09-30 (Europe) / 2025-10-08 (Americas)  
**Audience:** Experienced TrendMiner users   
**Goal:** Build an end-to-end OEE KPI reporting workflow using TrendMiner.

---




## About OEE
**Overall Equipment Effectiveness (OEE)** is the gold standard for measuring manufacturing productivity.
Simply put, it identifies the percentage of manufacturing time that is truly productive.

An OEE score of 100% means you are manufacturing only good parts, as fast as possible, with no stop time.
In OEE terms, that corresponds to:
- 100% Quality (only good parts)
- 100% Performance (as fast as possible)
- 100% Availability (no stop time)

**Why TrendMiner for OEE?**
TrendMiner provides a highly flexible approach to setting up OEE tracking systems, tightly connecting OEE metrics with production data for intuitive, two-way operations. Engineers can efficiently identify and categorize losses by analyzing trend data and leveraging analytics to calculate integral values promptly. Once collected, OEE data becomes a powerful resource for building reporting flows and performing retrospective analyses to highlight major losses and improvement opportunities.

## Solution training overview

This session is designed for experienced TrendMiner users who want to learn how to **build custom solution packages** on top of TrendMiner. Rather than being a theoretical exercise, this workshop mirrors the approach used by TrendMiner Data Analytics Engineers when implementing real-world solutions for customers. Although the outcome is a working OEE reporting solution, the real value is learning a **general solutions approach** that you can reuse for many other applications such as energy reporting, batch validation, and quality monitoring.

By the end of this training, you will have learned how to:

- Capture production events (batches for batch processes, operating periods for continuous processes) with monitors in TrendHub and display them as context items in a ContextHub view
- Load those context items via Python and write loss data to their context fields using the TrendMiner SDK
- Capture regular periods (weeks/months) and compute OEE KPIs (availability, productivity, quality, overall OEE) over those periods, using the data of production events captured earlier
- Creating actionable user dashboards based on this OEE data


### Prerequisites
- Access to your TrendMiner environment (URL, client credentials, and user credentials)
- Python with the TrendMiner SDK installed (wheel or pip package provided by TrendMiner)
- TrendMiner admin rights to create Context Types & Context Fields in ContextHub settings
- A process (batch or continuous) with tags you can read

## TrendMiner Python SDK setup
### Installation
Different versions of the TrendMiner SDK version are available on GitHub: <https://github.com/TrendMinerCS/sdk>

Uncomment one of the pip install commands bellow according to your TrendMiner version.

> **note:** if you have received a training account from the TrendMiner team you can use 2025.R2

In [ ]:
# Uncomment (remove the # before the line) the command according to your TrendMiner version:

# 2025R2
#!pip install https://github.com/TrendMinerCS/sdk/raw/main/sdk/2025R2/trendminer_interface-0.0.0-py3-none-any.whl

#2025R1
#!pip install https://github.com/TrendMinerCS/sdk/raw/main/sdk/2025R1/trendminer_interface-0.1.0.post161+86ad9f74.dirty-py3-none-any.whl

#2024R3.1
#!pip install https://github.com/TrendMinerCS/sdk/raw/main/sdk/2024R3.1/trendminer_interface-0.1.0.post160+1848287a.dirty-py3-none-any.whl

#2024R2.1
#!pip install https://github.com/TrendMinerCS/sdk/raw/main/sdk/2024R2.1/trendminer_interface-0.1.0.post162+bc00a459.dirty-py3-none-any.whl


### Install keyring
We will use keyring to securely store and retrive both our client secret and password.

In [ ]:
!pip install keyring

### Connect to TrendMiner
As a preparation we asked you to create a client and a local user (only applicable to SSO users). If you do not have any of these please ask for a training account.

#### Set TrendMiner credentials

Fill in your TrendMiner url, username and client id so we can later on use them to log into TrendMiner.

> **Note:** make sure that everything is between quotation marks ""

In [1]:
url = "YOUR TRENDMINER URL"
client_id = "YOUR TRENDMINER CLIENT ID"
username = "YOUR TRENDMINER USERNAME"

Now we will securely store your password and client secret using the keyring package.
The keyring package ensures that these credentials are stored safely by associating them with a unique combination of the TrendMiner URL and either your client ID or username.

**Instructions**:
1. Replace the placeholder values in the code below with your actual client secret and password.
2. Execute the cell to save your credentials securely in the keyring.
3. Once the cell has been executed successfully, remove the plain text values for both the client secret and password from the code to prevent them from being stored in clear text.

In [ ]:
import keyring

# Fill in the string bellow your client secret
# keyring.set_password(url, client_id, "YOUR CLIENT SECRET")

# Fill in the string bellow your password
# keyring.set_password(url, username, "YOUR PASSWORD")

#### Set timezone
You can specify your timezone to display all retrieved data in this format.

In [2]:
timezone = "Europe/Brussels"

#### Log into TrendMiner
In this step, we will use the credentials that were securely stored in the keyring to authenticate and establish a connection with TrendMiner.

1.	The cell retrieves the client secret and password from the keyring using the combination of the TrendMiner URL, client ID, and username.
2.	These credentials are then passed to the TrendMinerClient, along with the required connection parameters such as the URL and timezone.
3.	Once the connection is established, the client object can be used to interact with TrendMiner programmatically.
4.	The final line prints the client version to confirm that the connection has been created successfully.

In [3]:
from trendminer_interface import TrendMinerClient
import keyring

client = TrendMinerClient(
    url=url,
    client_id=client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz=timezone,
    verify=False,
)

print(client.version)

2025.R2.0-08


The print statement outputs the TrendMiner version and confirms that the client has been successfully created.

## Capturing production events
In this training, we will set up a monthly overview of OEE values. These monthly values will be aggregated from individual production events. Therefore, in a first stage, we will identify, capture and annotate individual production events as context items.

You should have already prepared an appropriate **Context Item Type** named `Production` with the following fields:
- `Performance Loss`
- `Quality Loss`

We can now create a **value-based search** in TrendMiner which captures such events. The basis for this search will simply be a condition that shows that our process is active.

If we are performing OEE on a **continuous** process, we already want to split our production events by the final reporting periods (months). We can do this by including the condition that our month tag (e.g. `TM_month_Europe_Brussels`) remains `constant` to our search.

For a **batch** process, we want to keep the individual batches events intact even if they fall within two different months. After all, there would otherwise not be a way to check whether those batches are taking too much time. We therefore only include the conditions that our batch process is active. We can deal with the problem of batches falling within two different months during the KPI calculation.

We will take a batch example, which is simply defined by the condition:
```
OEE-R1-active = 1
```

We now perform our search, and create (empty) `Production` **context items** from the search results. This takes care of the historic events. For a live solution, we need to make sure to also capture future events, which do by setting up a **monitor** which creates the same context items.

Finally, we want to capture these context items in dedicated **ContextHub views**. In these views, we should at least filter on:
- Timespan: the last x time, going back to include all historic context items
- context type: `Production`
- component: the tag, asset or attribute to which we have added our context items
- user: we do not want to capture someone else's events when working on the same server
- current state: we only want to capture `closed` context items

We can now save this ContextHub view in TrendMiner.

## Annotating production events
First step to annotating the new context items is pulling them into our coding environment. We can do this by simply getting the identifier from the ContextHub view url.

In [9]:
chv_production= client.context.view.from_identifier("0721cbe5-ee8b-4145-be93-558cc1f269a8")
df_production = chv_production.get_items()
df_production.head(3)  # Display the first 3 items

,key,identifier,identifier_external,description,type,component,created_by,created,last_modified,performance_loss,quality_loss
Production OEE annotated,,,,,,,,,,,
"[2025-01-01 01:00:00+01:00, 2025-01-01 06:39:00+01:00]",45C-6D,5cfc939a-7b5c-44ec-8c52-bc6544ed821f,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760085+02:00,2025-09-29 19:30:33.583506+02:00,NaN,NaN
"[2025-01-01 08:19:00+01:00, 2025-01-01 14:38:00+01:00]",45C-6C,ee0815ac-471f-4476-874c-f059446f8f2c,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760060+02:00,2025-09-29 19:30:33.625934+02:00,NaN,NaN
"[2025-01-01 17:26:00+01:00, 2025-01-01 22:49:00+01:00]",45C-6B,be978e72-214b-45ef-b971-53092fa430cc,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760033+02:00,2025-09-29 19:30:33.667154+02:00,NaN,NaN


We can now calculated performance and quality losses for our production events according to whatever logic we see fit. In our example, the performance loss is the duration by which a batch exceeds the target duration of 5h. The quality loss is dependent on the viscosity: whenever this value exceeds 140, the product is considered scrap, and we count the target duration of 5h as a quality loss. Note that this way, a batch can have both a quality and performance loss.

In [10]:
import pandas as pd

# Calculate performance loss
minimal_batch_duration = pd.Timedelta(hours=5)
df_production["performance_loss"] = (df_production.index.length - minimal_batch_duration).total_seconds()/60  # in minutes

# Calculate quality loss
viscosity_tag = client.tag.from_name("OEE-R1-viscosity")
df_production = df_production.interval.calculate(tag=viscosity_tag, operation="max", name="viscosity")
viscosity_too_high = df_production.pop("viscosity") > 140
quality_loss = viscosity_too_high * minimal_batch_duration.total_seconds()/60  # in minutes
df_production["quality_loss"] = quality_loss

We can display a few batches to see if the values we filled in make sense:

In [11]:
df_production.head(3)

,key,identifier,identifier_external,description,type,component,created_by,created,last_modified,performance_loss,quality_loss
Production OEE annotated,,,,,,,,,,,
"[2025-01-01 01:00:00+01:00, 2025-01-01 06:39:00+01:00]",45C-6D,5cfc939a-7b5c-44ec-8c52-bc6544ed821f,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760085+02:00,2025-09-29 19:30:33.583506+02:00,39.0,0.0
"[2025-01-01 08:19:00+01:00, 2025-01-01 14:38:00+01:00]",45C-6C,ee0815ac-471f-4476-874c-f059446f8f2c,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760060+02:00,2025-09-29 19:30:33.625934+02:00,79.0,0.0
"[2025-01-01 17:26:00+01:00, 2025-01-01 22:49:00+01:00]",45C-6B,be978e72-214b-45ef-b971-53092fa430cc,None,,production,OEE-R1-active,fvandael,2025-09-29 13:49:51.760033+02:00,2025-09-29 19:30:33.667154+02:00,23.0,0.0


Finally, we can update the production context items in TrendMiner. Note that every time we run this code, all context items will update. When we would set up an update service which runs regularly, it makes more sense to create a ContextHub view specifically to capture the context items with empty fields, and update only those.

In [12]:
df_production.context.update()

## Capturing monthly reporting periods
We can now capture the reporting periods in exactly the same way as the production events: create context items from a value-based search and a monitor on that search. 

A **Context Item Type** `OEE` with the following fields should already exist:
- `Availability`
- `Performance`
- `Quality`
- `OEE`


Capturing monthly periods with value-based search is easy:
```
TM_month_Europe_Brussels constant
```

Again, we should save a ContextHub view which captures (only) these context items, and record its identifier.

## Annotating monthly reports
We use our newly made ContextHub view to pull the monthly context items into TrendMiner:

In [63]:
chv_months = client.context.view.from_identifier("9126f17e-3883-4f08-8c2a-be5417a7f9ea")
df_months = chv_months.get_items()
df_months.head(3)

,key,identifier,identifier_external,description,type,component,created_by,created,last_modified,availability,OEE,performance,quality
Month OEE annotated,,,,,,,,,,,,,
"[2025-01-01 00:00:00+01:00, 2025-01-31 23:59:00+01:00]",45C-79,3ff187e6-a727-4585-9878-82d78ae01ad8,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723310+02:00,2025-09-29 14:49:47.723310+02:00,NaN,NaN,NaN,NaN
"[2025-02-01 00:00:00+01:00, 2025-02-28 23:59:00+01:00]",45C-78,26e121dd-4d5e-4c2b-b2ef-ce4121a09a8b,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723283+02:00,2025-09-29 14:49:47.723283+02:00,NaN,NaN,NaN,NaN
"[2025-03-01 00:00:00+01:00, 2025-03-31 23:59:00+02:00]",45C-77,d9f1d8fd-210f-479d-832e-e5dde07acc9a,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723257+02:00,2025-09-29 14:49:47.723257+02:00,NaN,NaN,NaN,NaN


We loop over the months one by one to calculate the availability, performance and quality parameters for each. Note that if the production events were not already loaded, we would need to retrieve them again from their ContextHub view.

In [64]:
for month in df_months.index:

    # Find the production events which fall within this month
    in_month = df_production.index.overlaps(month)
    in_month_production = df_production[in_month]

    # We need to account for the fact that production events can fall only partially within the month
    def get_in_month_ratio(event):
        event_cutoff_left = max(event.left, month.left)  # month or event start, whichever is last
        event_cutoff_right = min(event.right, month.right) # month or event end, whichever is first
        in_month_event_length = event_cutoff_right - event_cutoff_left 
        return in_month_event_length / event.length

    in_month_ratios = in_month_production.index.map(get_in_month_ratio)

    # The availability is the duration of production divided by the month duration
    total_production_duration = (in_month_production.index.length * in_month_ratios).sum()
    df_months.loc[month, "availability"] = total_production_duration / month.length

    # The performance is the target duration time divided by the actual duration
    total_performance_loss = pd.to_timedelta(
        (in_month_production["performance_loss"] * in_month_ratios).sum(), 
        unit="m",  # In our example, production losses were recorded in minutes
    )
    target_total_duration = total_production_duration - total_performance_loss
    df_months.loc[month, "performance"] = target_total_duration / total_production_duration

    # The quality is the % of the efficient production time was spent making accepted product
    total_quality_loss = pd.to_timedelta(
        (in_month_production["quality_loss"] * in_month_ratios).sum(),
        unit="m",  # In our example, production losses were recorded in minutes
    )
    df_months.loc[month, "quality"] = 1 - (total_quality_loss / target_total_duration)

We can calculate the overall OEE (A × P × Q) for all months at once:

In [68]:
df_months["OEE"] = df_months["availability"] * df_months["performance"] * df_months["quality"]

To avoid too many digits in ContextHub, we can round the numeric values to 4 digits:

In [71]:
df_months = df_months.round(4)

It makes sense to display some context items to make sure they contain sensible values:

In [75]:
df_months.head(3)

,key,identifier,identifier_external,description,type,component,created_by,created,last_modified,availability,OEE,performance,quality
Month OEE annotated,,,,,,,,,,,,,
"[2025-01-01 00:00:00+01:00, 2025-01-31 23:59:00+01:00]",45C-79,3ff187e6-a727-4585-9878-82d78ae01ad8,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723310+02:00,2025-09-29 14:49:47.723310+02:00,0.7337,0.5240,0.8242,0.8666
"[2025-02-01 00:00:00+01:00, 2025-02-28 23:59:00+01:00]",45C-78,26e121dd-4d5e-4c2b-b2ef-ce4121a09a8b,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723283+02:00,2025-09-29 14:49:47.723283+02:00,0.7281,0.4913,0.8076,0.8355
"[2025-03-01 00:00:00+01:00, 2025-03-31 23:59:00+02:00]",45C-77,d9f1d8fd-210f-479d-832e-e5dde07acc9a,None,,OEE_report,TM_month_Europe_Brussels,fvandael,2025-09-29 14:49:47.723257+02:00,2025-09-29 14:49:47.723257+02:00,0.7347,0.5367,0.8129,0.8986


Finally, we can now update the context items in TrendMiner, resulting in a nice monthly overview of our OEE KPIs

In [78]:
df_months.context.update()

## Creating dashboards
We can now build a custom dashboards around our OEE data, showing our monthly KPIs and potentially the production events as well. **Conditional formatting** in ContextHub can make it very clear how we have been performing.

If you have access, MLHub can be used to create custom visualizations. We can load ContextHub views into MLHub and extract the OEE metrics from the views. The metrics can then be deployed to a custom visualization. For example, we can create multi-line charts, bar charts, and radar plots of the OEE metrics in MLHub. Then we create a pipeline of the visualization to display on a DashHub dashboard.